In [ ]:
# 💣 Instala todo desde cero, limpio y probado
!pip install numpy==1.23.5 scipy==1.9.3 scikit-learn==1.1.3 rembg[gpu]==2.0.50 faiss-cpu opencv-python-headless colormath tqdm --force-reinstall
import os; os.kill(os.getpid(), 9)  # Reinicio obligatorio


  Using cached numpy-1.23.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.3 kB)
  Using cached scipy-1.9.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
  Using cached rembg-2.0.50-py3-none-any.whl.metadata (14 kB)
  Using cached faiss_cpu-1.11.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (4.8 kB)
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (20 kB)
  Using cached colormath-3.0.0-py3-none-any.whl
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached onnxruntime-1.22.0-cp311-cp311-manylinux_2_27_x86_64.manylinux_2_28_x86_64.whl.metadata (4.5 kB)
  Using cached pillow-11.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached pooch-1.8.2-py3-none-any.whl.metadata (10 kB)
  Using cached PyMatting-1.1.13-py3-none-any.whl.metadata (7.5 kB)
  Using cached scikit_image-0.25.2-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86

In [ ]:
import numpy as np
print(np.__version__)

if not hasattr(np, 'asscalar'):
    np.asscalar = lambda array: array.item()

2.0.2


In [ ]:
# Celda 0 ────────────────────────────────────────────────────────────────
from google.colab import drive
drive.mount('/content/drive')

BASE_DIR  = '/content/drive/MyDrive/IABD/Recomendacion'   # carpeta de tu proyecto
IMG_ROOTS = [                                        # TUS tres carpetas reales
    '/content/drive/MyDrive/IABD/archive/images_new',
    '/content/drive/MyDrive/IABD/archive/images_new2',
    '/content/drive/MyDrive/IABD/archive/images_original',
]
FEAT_DIR  = f'{BASE_DIR}/features'                   # Parquet + índice FAISS
!mkdir -p "{FEAT_DIR}"


Mounted at /content/drive


In [ ]:
# Celda 1 (ACTUALIZADA) ─────────────────────────────────────────────────
!pip install -q rembg[gpu] faiss-cpu opencv-python-headless scikit-learn colormath tqdm


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.5 MB/s eta 0:00:00


In [ ]:
# Celda 2
import cv2, faiss, gc, os
import numpy as np, pandas as pd
from pathlib import Path
from tqdm.auto import tqdm
from rembg import remove
from sklearn.cluster import KMeans
from colormath.color_objects import LabColor
from colormath.color_diff import delta_e_cie2000

def dominant_lab(img_bgr, mask=None, k=3):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    if mask is not None:
        lab = lab[mask > 0]
    km  = KMeans(k, n_init=10).fit(lab.reshape(-1,3))  # ← cambio aquí
    dom = km.cluster_centers_[np.bincount(km.labels_).argmax()]
    return dom.astype(np.float32)

def lab_to_rgb(lab):
    lab = np.clip(lab, 0, 255).astype('uint8').reshape(1,1,3)
    rgb = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)[0,0,::-1]
    return tuple(int(c) for c in rgb)


In [ ]:
# Celda 3 ────────────────────────────────────────────────────────────────
# 3.1 Recolectar TODAS las imágenes (extensiones .jpg, .jpeg, .png)
EXTS = ('*.jpg', '*.jpeg', '*.png')
catalog_paths = []
for root in IMG_ROOTS:
    for ext in EXTS:
        catalog_paths += list(Path(root).rglob(ext))
catalog_paths = sorted(catalog_paths)

print(f'Encontradas {len(catalog_paths)} imágenes en las tres carpetas')

# 3.2 Procesar catálogo
records = []
for path in tqdm(catalog_paths, desc='Procesando catálogo'):
    img  = cv2.imread(str(path))
    mask = remove(img, only_mask=True)             # quitar fondo
    dom  = dominant_lab(img, mask=mask, k=3)       # color dominante

    # garment_id único = ruta relativa sin extensión, con barras → “_”
    rel_path  = Path(os.path.relpath(path, BASE_DIR))
    garment_id = str(rel_path.with_suffix('')).replace(os.sep, '_')

    records.append({
        'garment_id': garment_id,
        'L': dom[0], 'a': dom[1], 'b': dom[2],
        'path': str(path)
    })
    del img, mask; gc.collect()

color_df = pd.DataFrame(records)
color_df.to_parquet(f'{FEAT_DIR}/catalog_lab.parquet')
print('→ Guardados', len(color_df), 'vectores de color en Parquet')


Encontradas 6609 imágenes en las tres carpetas


Procesando catálogo:   0%|          | 0/6609 [00:00<?, ?it/s]


  0%|                                               | 0.00/176M [00:00<?, ?B/s]
  3%|▉                                     | 4.45M/176M [00:00<00:03, 44.4MB/s]
  7%|██▊                                   | 13.2M/176M [00:00<00:02, 69.6MB/s]
 12%|████▋                                 | 21.5M/176M [00:00<00:02, 76.0MB/s]
 17%|██████▍                               | 30.0M/176M [00:00<00:01, 79.5MB/s]
 22%|████████▎                             | 38.6M/176M [00:00<00:01, 81.7MB/s]
 27%|██████████▏                           | 47.0M/176M [00:00<00:01, 82.7MB/s]
 31%|███████████▉                          | 55.3M/176M [00:00<00:01, 81.9MB/s]
 36%|█████████████▋                        | 63.5M/176M [00:00<00:01, 81.4MB/s]
 41%|███████████████▌                      | 71.8M/176M [00:00<00:01, 82.0MB/s]
 46%|█████████████████▎                    | 80.1M/176M [00:01<00:01, 82.2MB/s]
 50%|███████████████████▏                  | 88.7M/176M [00:01<00:01, 83.3MB/s]
 55%|█████████████████████             

→ Guardados 6609 vectores de color en Parquet


In [ ]:
# Celda 4 ────────────────────────────────────────────────────────────────
vecs  = color_df[['L','a','b']].to_numpy(dtype='float32')
index = faiss.IndexFlatL2(3)
index.add(vecs)
faiss.write_index(index, f'{FEAT_DIR}/faiss_lab.idx')
print('→ Índice FAISS creado con', index.ntotal, 'vectores')


NameError: name 'color_df' is not defined

In [ ]:
# Celda 5 ────────────────────────────────────────────────────────────────
color_df = pd.read_parquet(f'{FEAT_DIR}/catalog_lab.parquet')
index    = faiss.read_index(f'{FEAT_DIR}/faiss_lab.idx')

def recommend_by_color(img_path, top_k=8):
    img   = cv2.imread(img_path)
    mask  = remove(img, only_mask=True)
    q_lab = dominant_lab(img, mask=mask, k=3)

    # Búsqueda rápida (L2 en Lab 3-D)
    D, I  = index.search(q_lab.reshape(1,3).astype('float32'), top_k*3)

    # Refinamiento perceptual ΔE00
    idxs, dists = I[0], []
    for j in idxs:
        c = color_df.iloc[j][['L','a','b']].values
        dists.append(delta_e_cie2000(LabColor(*q_lab), LabColor(*c)))

    best = (color_df.iloc[idxs]
            .assign(deltaE=dists)
            .sort_values('deltaE')
            .head(top_k)
            .reset_index(drop=True))
    return q_lab, best


In [ ]:
# Celda 6 ────────────────────────────────────────────────────────────────
TEST_IMG = '/content/drive/MyDrive/IABD/archive/images_original/4e0f02de-645d-4615-a13b-749d8027c85b.jpg'   # ← pon aquí una imagen cualquiera

q_lab, resultados = recommend_by_color(TEST_IMG, top_k=6)
print('Color dominante (L,a,b):', q_lab)
display(resultados[['garment_id', 'deltaE']])

# Cuadradito del color dominante
import matplotlib.pyplot as plt
rgb = lab_to_rgb(q_lab)
plt.figure(figsize=(1,1)); plt.axis('off')
plt.imshow([[(rgb[0]/255, rgb[1]/255, rgb[2]/255)]])
plt.show()

# Miniaturas de las recomendaciones
from IPython.display import Image, display
for p in resultados['path']:
    display(Image(p, width=140))


Output hidden; open in https://colab.research.google.com to view.